In [1]:
# ONLY FOR GOOGLE COLAB
from google.colab import drive
import os
drive.mount('/content/drive')
WORKING_DIRECTORY = '/content/drive/My Drive/hack_october/Dataset'

os.chdir(WORKING_DIRECTORY)

Mounted at /content/drive


# Dataset creation
Based on the reviewed data, we have defined a set of features that may be useful in determining the quality of markup.

Here we add this features to pandas.DataFrame and export it.

## Features
iou	iou_intersections	center	ml_count	expert_count	expert_intersect_area	expert_union_area	expert_area	ml_intersect_area	ml_union_area	ml_area
1. IoU
2. IoU minus intersections
3. Distance between centers of the figures
4. Count of blobs recognized by AI
5. Count of blobs recognized by expert
6. Area of blobs' intersections of blobs recognized by expert
7. Area of blobs' union of blobs recognized by expert
8. Sum of areas of all blobs recognized by expert
9. Area of blobs' intersections of blobs recognized by AI
10. Area of blobs' union of blobs recognized by AI
11. Sum of areas of all blobs recognized by AI

In [2]:
import pandas as pd
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

## Classes
They can create an opencv Mat oblect and calculate the area.

In [3]:
class Rectangle:
    def __init__(self, xcenter, ycenter, rhorizontal, rvertical):
        self.xcenter = xcenter
        self.ycenter = ycenter
        self.rhorizontal = rhorizontal
        self.rvertical = rvertical
    
    def opencv(self, thickness):
        image = np.zeros((1024,1024), np.uint8)
        pt1 = (int(self.xcenter-self.rhorizontal), int(self.ycenter-self.rvertical))
        pt2 = (int(self.xcenter+self.rhorizontal), int(self.ycenter+self.rvertical))
        cv2.rectangle(image, pt1, pt2, 255, thickness)
        return image
    
    def area(self):
        return cv2.countNonZero(self.opencv(-1))

class Circle:
    def __init__(self, xcenter, ycenter, rhorizontal, rvertical):
        self.xcenter = xcenter
        self.ycenter = ycenter
        self.rhorizontal = rhorizontal
        self.rvertical = rvertical
    
    def opencv(self, thickness):
        image = np.zeros((1024,1024), np.uint8)
        cv2.ellipse(image, (int(self.xcenter), int(self.ycenter)), (int(self.rhorizontal), int(self.rvertical)), 0, 0, 360, 255, thickness)
        return image
    
    def area(self):
        return cv2.countNonZero(self.opencv(-1))

## Functions
Here you can see some functions taht used for extraction of our feaures.

In [19]:
def draw(photo, un):  # This function draw union between all blobs on picture.
    im = np.zeros((1024,1024), np.uint8)
    user = photo[photo[' user_name'] == un]
    for _, u in user.iterrows():
        if u[' shape'] == 'rectangle':
            fig = Rectangle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1)
        if u[' shape'] == 'circle':
            fig = Circle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1)
        im = cv2.bitwise_or(im, fig)
    return im

def area(photo, un):  # This function computes the area of all blobs.
    im = np.zeros((1024,1024), np.uint8)
    user = photo[photo[' user_name'] == un]
    if len(user) > 0:
        ar = 0.0
        for _, u in user.iterrows():
            if u[' shape'] == 'rectangle':
                fig = Rectangle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical'])
            elif u[' shape'] == 'circle':
                fig = Circle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical'])
            ar += fig.area()
        return ar
    else:
        return 0.0

def area_union(photo, un):  # This function computes the area of blobs' union.
    im = np.zeros((1024,1024), np.uint8)
    user = photo[photo[' user_name'] == un]
    if len(user) > 0:
        for _, u in user.iterrows():
            if u[' shape'] == 'rectangle':
                fig = Rectangle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1)
            if u[' shape'] == 'circle':
                fig = Circle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1)
            im = cv2.bitwise_or(im, fig)
        return cv2.countNonZero(im)
    else:
        return 0.0

def area_intersection(photo, un):  # This function computes the area of blobs' intersection.
    im = np.zeros((1024,1024), np.uint8)
    user = photo[photo[' user_name'] == un]
    figs = []
    ar = 0.0
    if len(user) > 0:
        for _, u in user.iterrows():
            if u[' shape'] == 'rectangle':
                figs.append(Rectangle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1))
            if u[' shape'] == 'circle':
                figs.append(Circle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1))
        for i in range(len(figs) - 2):
            for j in range(i+1, len(figs)):
                ar += cv2.countNonZero(cv2.bitwise_and(figs[i], figs[j]))
        return ar
    else:
        return 0.0

def draw_intersections(photo, un):  # This function draws the blobs' intersection.
    im = np.zeros((1024,1024), np.uint8)
    user = photo[photo[' user_name'] == un]
    figs = []
    for _, u in user.iterrows():
        if u[' shape'] == 'rectangle':
            figs.append(Rectangle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1))
            # draw_rectangle(im, u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical'])
        if u[' shape'] == 'circle':
            figs.append(Circle(u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical']).opencv(-1))
            # draw_ellipse(im, u[' xcenter'], u[' ycenter'], u[' rhorizontal'], u[' rvertical'])
    im = sum(figs)

    intersections = 0
    for i in range(len(figs) - 1):
        for j in range(i+1, len(figs)):
            intersections += cv2.countNonZero(cv2.bitwise_and(figs[i], figs[j]))

    return im, intersections

def iou(one, two):  # Calculating IoU
    band = cv2.bitwise_and(one, two)
    bor = cv2.bitwise_or(one, two)
    try:
        return cv2.countNonZero(band) / cv2.countNonZero(bor)
    except ZeroDivisionError:
        return 0.0

def check_intersect(one, two):
    band = cv2.bitwise_and(one, two)
    return cv2.countNonZero(band) > 0

def iou_intersections(one, two, intersections):  # Calculating IoU with penalty for intersections
    band = cv2.bitwise_and(one, two)
    bor = cv2.bitwise_or(one, two)
    try:
        return cv2.countNonZero(band) / (cv2.countNonZero(bor) + intersections)
    except ZeroDivisionError:
        return 0.0

## Import data

In [20]:
data = pd.read_csv("DX_TEST_RESULT_FULL.csv")

In [41]:
# data = data.drop(["Unnamed: 0"], axis=1)

In [21]:
data

,file_name,user_name,xcenter,ycenter,rhorizontal,rvertical,shape,Unnamed: 7
0,00000150_002,sample_2,737.882353,690.389140,64.482655,28.573152,rectangle,NaN
1,00000181_061,sample_2,317.007541,749.466063,133.984917,198.081448,rectangle,NaN
2,00000181_061,sample_2,731.318250,926.696833,152.132730,69.502262,rectangle,NaN
3,00000181_061,sample_2,714.714932,628.609351,129.351433,216.229261,rectangle,NaN
4,00000211_019,sample_2,360.639517,589.224736,105.025641,219.318250,rectangle,NaN
...,...,...,...,...,...,...,...,...
1062,00020000_000,sample_3,314.595556,146.526823,44.373333,14.222222,rectangle,NaN
1063,00020124_003,sample_3,414.720000,352.464601,53.475556,67.697778,rectangle,NaN
1064,00020124_003,sample_3,297.358229,632.225191,122.311111,51.768889,rectangle,NaN
1065,00020124_003,sample_3,414.720000,352.464601,53.475556,67.697778,rectangle,NaN


In [22]:
columns = ['file_name', 'user_name']
for i in range(23):
    columns += ['expert_xcenter_' + str(i), 'expert_ycenter_' + str(i), 'expert_rhorizontal_' + str(i), 'expert_rvertical_' + str(i), 'expert_shape_' + str(i)]

for i in range(23):
    columns += ['xcenter_' + str(i), 'ycenter_' + str(i), 'rhorizontal_' + str(i), 'rvertical_' + str(i), 'shape_' + str(i)]


In [23]:
features = pd.DataFrame(columns=columns)

In [24]:
for fn in pd.unique(data['file_name']):
    # print('row')
    for s in ['sample_1', 'sample_2', 'sample_3']:
        # print(s)
        cols = [fn, s]
        i = 0
        while i < 23:
            if len(data[(data['file_name']==fn) & (data[' user_name']=='Expert')]):
                for index, e in data[(data['file_name']==fn) & (data[' user_name']=='Expert')].iterrows():
                    cols += [e[2], e[3], e[4], e[5], e[6]]
                    i += 1
                    # print(i)
                    if i >= 23:
                        # print('break')
                        break
            else:
                cols += [0, 0, 0, 0, 0]
                i += 1
        # print('whbreak')
        # print(len(cols))
        i = 0
        while i < 23:
            if len(data[(data['file_name']==fn) & (data[' user_name']==s)]):
                for index, e in data[(data['file_name']==fn) & (data[' user_name']==s)].iterrows():
                    cols += [e[2], e[3], e[4], e[5], e[6]]
                    i += 1
                    # print(i)
                    if i >= 23:
                        break
            else:
                cols += [0, 0, 0, 0, 0]
                i += 1
        # print(len(cols))
        
        cols = pd.DataFrame(np.array([cols]), columns=columns)
        features = pd.concat((features, cols), ignore_index=True)

In [25]:
features = features[(features['shape_0'] != '0') & (features['expert_shape_0'] != '0')].reset_index(drop=True)

In [26]:
features

,file_name,user_name,expert_xcenter_0,expert_ycenter_0,expert_rhorizontal_0,expert_rvertical_0,expert_shape_0,expert_xcenter_1,expert_ycenter_1,expert_rhorizontal_1,expert_rvertical_1,expert_shape_1,expert_xcenter_2,expert_ycenter_2,expert_rhorizontal_2,expert_rvertical_2,expert_shape_2,expert_xcenter_3,expert_ycenter_3,expert_rhorizontal_3,expert_rvertical_3,expert_shape_3,expert_xcenter_4,expert_ycenter_4,expert_rhorizontal_4,expert_rvertical_4,expert_shape_4,expert_xcenter_5,expert_ycenter_5,expert_rhorizontal_5,expert_rvertical_5,expert_shape_5,expert_xcenter_6,expert_ycenter_6,expert_rhorizontal_6,expert_rvertical_6,expert_shape_6,expert_xcenter_7,expert_ycenter_7,expert_rhorizontal_7,...,xcenter_15,ycenter_15,rhorizontal_15,rvertical_15,shape_15,xcenter_16,ycenter_16,rhorizontal_16,rvertical_16,shape_16,xcenter_17,ycenter_17,rhorizontal_17,rvertical_17,shape_17,xcenter_18,ycenter_18,rhorizontal_18,rvertical_18,shape_18,xcenter_19,ycenter_19,rhorizontal_19,rvertical_19,shape_19,xcenter_20,ycenter_20,rhorizontal_20,rvertical_20,shape_20,xcenter_21,ycenter_21,rhorizontal_21,rvertical_21,shape_21,xcenter_22,ycenter_22,rhorizontal_22,rvertical_22,shape_22
0,00000150_002,sample_1,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,...,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle
1,00000150_002,sample_2,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,...,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle
2,00000150_002,sample_3,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.1

In [27]:
for fn in pd.unique(data['file_name']):
    photo = data[data['file_name']==fn]
    expert = draw(photo, 'Expert')
    samples = ['sample_1', 'sample_2', 'sample_3']
    for sample in samples:
        current = draw(photo, sample)
        features.loc[(features['file_name']==fn) & (features['user_name']==sample), "has_intersections"] = check_intersect(current, expert)
        features.loc[(features['file_name']==fn) & (features['user_name']==sample), "iou"] = iou(current, expert)
        # print(sample, iou(current, expert))

In [28]:
for fn in pd.unique(data['file_name']):
    photo = data[data['file_name']==fn]
    expert = draw(photo, 'Expert')
    samples = ['sample_1', 'sample_2', 'sample_3']
    for sample in samples:
        current, inters = draw_intersections(photo, sample)
        features.loc[(features['file_name']==fn) & (features['user_name']==sample), "iou_intersections"] = iou_intersections(current, expert, inters)

In [29]:
def point_distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

def distance_between_centers_feature(filename, user_name):
    doc_markups = data[(data[" user_name"] == "Expert") & (data["file_name"] == filename)]
    network_markups = data[(data[" user_name"] == user_name) & (data["file_name"] == filename)]
        
    sum_distances = 0
    for _, net_m in network_markups.iterrows():
        min_distance = 10000000000000
        for _, doc_m in doc_markups.iterrows():
            min_distance = min(min_distance, point_distance((net_m[2], net_m[3]), (doc_m[2], doc_m[3])))
        if min_distance == 10000000000000:
            print("FUCK!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        sum_distances += min_distance**2
        
    if len(network_markups) != 0:
        sum_distances = sum_distances / (1448 * len(network_markups))**2

    return sum_distances

In [ ]:
for fn in pd.unique(data['file_name']):
    samples = ['sample_1', 'sample_2', 'sample_3']
    for sample in samples:
        features.loc[(features['file_name']==fn) & (features['user_name']==sample), "center"] = distance_between_centers_feature(fn, sample)

In [31]:
for fn in pd.unique(data['file_name']):
    samples = ['sample_1', 'sample_2', 'sample_3']
    for sample in samples:
        features.loc[(features['file_name']==fn) & (features['user_name']==sample), "ml_count"] = len(data[(data['file_name']==fn) & (data[' user_name']==sample)])
        features.loc[(features['file_name']==fn) & (features['user_name']==sample), "expert_count"] = len(data[(data['file_name']==fn) & (data[' user_name']=="Expert")])

In [32]:
for fn in pd.unique(data['file_name']):
    samples = ['sample_1', 'sample_2', 'sample_3']
    for sample in samples:
        photo = data[data['file_name']==fn]
        expert = area(photo, 'Expert')
        expert_i = area_intersection(photo, 'Expert')
        expert_u = area_union(photo, 'Expert')
        for sample in samples:
            current = area(photo, sample)
            current_i = area_intersection(photo, sample)
            current_u = area_union(photo, sample)

            features.loc[(features['file_name']==fn) & (features['user_name']==sample), "expert_intersect_area"] = expert_i
            features.loc[(features['file_name']==fn) & (features['user_name']==sample), "expert_union_area"] = expert_u
            features.loc[(features['file_name']==fn) & (features['user_name']==sample), "expert_area"] = expert

            features.loc[(features['file_name']==fn) & (features['user_name']==sample), "ml_intersect_area"] = current_i
            features.loc[(features['file_name']==fn) & (features['user_name']==sample), "ml_union_area"] = current_u
            features.loc[(features['file_name']==fn) & (features['user_name']==sample), "ml_area"] = current


In [33]:
features

,file_name,user_name,expert_xcenter_0,expert_ycenter_0,expert_rhorizontal_0,expert_rvertical_0,expert_shape_0,expert_xcenter_1,expert_ycenter_1,expert_rhorizontal_1,expert_rvertical_1,expert_shape_1,expert_xcenter_2,expert_ycenter_2,expert_rhorizontal_2,expert_rvertical_2,expert_shape_2,expert_xcenter_3,expert_ycenter_3,expert_rhorizontal_3,expert_rvertical_3,expert_shape_3,expert_xcenter_4,expert_ycenter_4,expert_rhorizontal_4,expert_rvertical_4,expert_shape_4,expert_xcenter_5,expert_ycenter_5,expert_rhorizontal_5,expert_rvertical_5,expert_shape_5,expert_xcenter_6,expert_ycenter_6,expert_rhorizontal_6,expert_rvertical_6,expert_shape_6,expert_xcenter_7,expert_ycenter_7,expert_rhorizontal_7,...,rhorizontal_17,rvertical_17,shape_17,xcenter_18,ycenter_18,rhorizontal_18,rvertical_18,shape_18,xcenter_19,ycenter_19,rhorizontal_19,rvertical_19,shape_19,xcenter_20,ycenter_20,rhorizontal_20,rvertical_20,shape_20,xcenter_21,ycenter_21,rhorizontal_21,rvertical_21,shape_21,xcenter_22,ycenter_22,rhorizontal_22,rvertical_22,shape_22,has_intersections,iou,iou_intersections,center,ml_count,expert_count,expert_intersect_area,expert_union_area,expert_area,ml_intersect_area,ml_union_area,ml_area
0,00000150_002,sample_1,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,...,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,772.509844559585,697.1689119170983,18.039378238341893,16.978238341968847,circle,True,0.351411,0.351411,0.000007,1.0,1.0,0.0,2729.0,2729.0,0.0,959.0,959.0
1,00000150_002,sample_2,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,...,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,737.8823529411765,690.3891402714933,64.48265460030166,28.57315233785823,rectangle,True,0.306655,0.306655,0.000588,1.0,1.0,0.0,2729.0,2729.0,0.0,7540.0,7540.0
2,00000150_002,sample_3,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.198391420911552,28.13941018766758,circle,771.4316353887401,700.7399463806971,30.1983

In [55]:
features.to_csv('wo_aug.csv')